In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Dropout,
    BatchNormalization,
)
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# 加载MNIST数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 数据预处理
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

x_train = np.expand_dims(x_train, axis=-1)  # 增加通道维度
x_test = np.expand_dims(x_test, axis=-1)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 数据增强
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    zoom_range=0.5,
    shear_range=0.5,
    fill_mode="nearest",
)
datagen.fit(x_train)


# 构建改进的CNN模型
# 卷积神经网络
model = Sequential(
    [
        # 卷积层
        Conv2D(32, (3, 3), activation="relu", input_shape=(28, 28, 1), padding="same"),
        BatchNormalization(),

        # 连续做两次 两个卷积层相同的操作 可以帮助模型更好地提取和学习特征
        Conv2D(32, (3, 3), activation="relu", padding="same"),
        BatchNormalization(),
        MaxPooling2D((2, 2)), # 下采样，减少特征图尺寸 参数为池化窗口大小
        Dropout(0.25), # 平衡正则化和模型性能
        
        Conv2D(64, (3, 3), activation="relu", padding="same"),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        Conv2D(128, (3, 3), activation="relu", padding="same"),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.4),
        
        # 平坦层
        Flatten(), 
        
        # ------------- 输出 ----------------------
        
        # 全连接神经网络 输出结论
        Dense(128, activation="relu"),
        BatchNormalization(),
        Dropout(0.5), 

        Dense(10, activation="softmax"),
    ]
)

# 编译模型
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), # 学习率
    loss="categorical_crossentropy", # 损失函数
    metrics=["accuracy"],
)

# 回调函数
# 在损失函数平稳时减小学习率
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=8, min_lr=1e-6, verbose=1
)

# 提前停止 达到最优值时停止训练
early_stopping = EarlyStopping(
    monitor="val_loss", patience=15, restore_best_weights=True, verbose=1
)


# 训练模型
model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    epochs=200,
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // 64,
    callbacks=[reduce_lr, early_stopping],
)

# 评估模型
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")

# 保存模型
model.save("mnist_model_ubuntu.keras")

2025-01-14 15:17:49.323196: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2025-01-14 15:17:50.880005: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2025-01-14 15:17:50.880953: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2025-01-14 15:17:50.887065: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:af:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.75GiB deviceMemoryBandwidth: 573.69GiB/s
2025-01-14 15:17:50.887084: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2025-01-14 15:17:50.888668: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.

Epoch 1/200


2025-01-14 15:17:52.426049: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2025-01-14 15:17:52.670778: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


937/937 [==============================] - 32s 32ms/step - loss: 2.1569 - accuracy: 0.3431 - val_loss: 0.1622 - val_accuracy: 0.9548
Epoch 2/200
937/937 [==============================] - 30s 32ms/step - loss: 0.8402 - accuracy: 0.7309 - val_loss: 0.1315 - val_accuracy: 0.9574
Epoch 3/200
937/937 [==============================] - 28s 30ms/step - loss: 0.6233 - accuracy: 0.8046 - val_loss: 0.0871 - val_accuracy: 0.9737
Epoch 4/200
937/937 [==============================] - 30s 32ms/step - loss: 0.5122 - accuracy: 0.8357 - val_loss: 0.1037 - val_accuracy: 0.9683
Epoch 5/200
937/937 [==============================] - 30s 32ms/step - loss: 0.4567 - accuracy: 0.8573 - val_loss: 0.0699 - val_accuracy: 0.9771
Epoch 6/200
937/937 [==============================] - 30s 32ms/step - loss: 0.4125 - accuracy: 0.8687 - val_loss: 0.0402 - val_accuracy: 0.9875
Epoch 7/200
937/937 [==============================] - 29s 31ms/step - loss: 0.3917 - accuracy: 0.8763 - val_loss: 0.0391 - val_accuracy: 0.98